**Use GPU for better performance**


Credits:

* [qweasdd/manga-colorization-v2](https://github.com/qweasdd/manga-colorization-v2)

* [Keiser04/manga-colorization-v2](https://github.com/Keiser04/manga-colorization-v2)

* [xinntao/Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN/)

Colab By:

[Swakshan/AI-Manga-Colorizer](https://github.com/Swakshan/AI-Manga-Colorizer)

**Version 2**

Changelogs:

* ++ Download using url
* ++ Image Compression

In [ ]:
#@title Requirements
from IPython.display import clear_output

try:
  !pip install patool
  !pip install torchvision
  !pip install opencv-python
  !pip install python-telegram-bot
  clear_output()
except Exception as e:
  print(str(e))
print("DONE: Installed Requirements")

DONE: Installed Requirements


In [ ]:
#@title Common

import os,requests
from bs4 import BeautifulSoup
import requests
import os
from urllib.parse import urlparse
from IPython.display import clear_output
from tqdm import tqdm
from zipfile import ZipFile 


PROJECT_NAME = "manga-colorization-v2"

MANGA_FOLDER = "/content/Manga"



def printLine():
  print("--------------------------")

def downloader(path,url,hdr={}):
  if os.path.exists(path):
    print(f"Exists: {path}")
    return True

  # print(f"Downloading {name}")
  r = requests.get(url,headers=hdr)
  total = int(r.headers.get('content-length', 0))
  if r.status_code == 200:
    with open(path, 'wb') as f,tqdm(
        desc=path,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    )as bar:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                size = f.write(chunk)
                bar.update(size)
    print(f"Downloaded: {path}")
    return
  else:
    print(f"Error Downloading : {path}")
    return False

def mkdir(path):
  try:
    os.makedirs(path)
    print(f"CREATED: {path}")
  except Exception as e:
    print(str(e))

def get_file_size_mb(file_path):
    # Get the size of the file in bytes
    size_in_bytes = os.path.getsize(file_path)
    # Convert bytes to megabytes (1 MB = 1024 * 1024 bytes)
    size_in_mb = size_in_bytes / (1024 * 1024)
    # print(size_in_mb)
    size_in_mb = round(size_in_mb,2)
    return f"{size_in_mb}MB"


def unzip(inp_dir,out_dir):
  try:
    with ZipFile(inp_dir, 'r') as zObject: 
        zObject.extractall(path=out_dir)
    print(f"UNZIPPED: {out_dir}")
  except Exception as e:
    print(str(e))

  

mkdir(MANGA_FOLDER)

CREATED: /content/Manga


In [ ]:
#@title Download Project Files
import os
from IPython.display import clear_output

#@markdown Clone Keiser04/manga-colorization-v2
project = True # @param {type:"boolean"}
generator = True # @param {type:"boolean"}
generator_model_url = "https://drive.google.com/file/d/1qmxUEKADkEM4iYLp1fpPLLKnfZ6tcF-t/view?usp=sharing" # @param {type:"string"}
extractor = True # @param {type:"boolean"}
extractor_model_url = "https://drive.google.com/file/d/12cbNyJcCa1zI2EBz6nea3BXl21Fm73Bt/view?usp=sharing" # @param {type:"string"}
denoiser = True # @param {type:"boolean"}
denoiser_model_url = "https://drive.google.com/uc?export=download&id=161oyQcYpdkVdw8gKz_MA8RD-Wtg9XDp3" # @param {type:"string"}
#@markdown Clone xinntao/Real-ESRGAN
upscaler = True # @param {type:"boolean"}
upscaler_model_url = "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth" # @param {type:"string"}
upscaler_model_name = "RealESRGAN_x4plus_anime_6B.pth" # @param {type:"string"}

G_PATH = f"{PROJECT_NAME}/networks/"
D_PATH = f"{PROJECT_NAME}/denoising/models/"
done_str = ""
if project:
  !git clone https://github.com/Keiser04/manga-colorization-v2.git
  done_str+=" "+PROJECT_NAME
if generator:
  mkdir(G_PATH)
  downloader(f"{G_PATH}generator.zip",generator_model_url)
  done_str+=" generator.zip"
if extractor:
  print(extractor_model_url)
  mkdir(G_PATH)
  downloader(f"{G_PATH}extractor.pth",extractor_model_url)
  done_str+=" extractor.pth"
if denoiser:
  mkdir(D_PATH)
  downloader(f"{D_PATH}net_rgb.pth",denoiser_model_url)
  done_str+=" net_rgb.pth"


if upscaler:
  !git clone https://github.com/xinntao/Real-ESRGAN.git
  %cd Real-ESRGAN
  # Set up the environment
  !pip install basicsr facexlib gfpgan
  !pip install -r requirements.txt
  !python setup.py develop
  # Download the pre-trained model
  downloader(f"weights/{upscaler_model_name}",upscaler_model_url)
  # !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
  %cd ..
  done_str+=" upscaler"

clear_output()
print("DONE:"+done_str)

DONE: generator.zip extractor.pth net_rgb.pth


#Manga

In [ ]:
#@title Upload Manga
#@markdown supports zip,cbr formats
from google.colab import files
import patoolib,os
from IPython.display import clear_output

try:
  done_str = ""
  uploaded  = files.upload()
  for fn in uploaded.keys():
      manga_name = input("manga name ?: ")
      chapter_num = input("Chapter number?: ")

      manga_name = "random" if not len(manga_name) else manga_name
      chapter_num = "Chapter 1" if not len(chapter_num) else f"Chapter {chapter_num}"

      path = os.path.join(MANGA_FOLDER,f"{manga_name}/{chapter_num}")
      mkdir(path)

      patoolib.extract_archive(archive=fn,verbosity=1,outdir=path)
      cur_done = f"DONE: {path}"
      clear_output()
      print(cur_done)
      done_str +=f"{cur_done}\n"
except Exception as e:
  print(str(e))

print(done_str)


In [ ]:
#@title Download zip,cbr
url = "" # @param {type:"string"}
manga_name = "Kagurabachi" # @param {type:"string"}
chapter_num = "23" # @param {type:"string"}
import os
import patoolib

  

def download_zip():
  if len(url) and len(manga_name) and len(chapter_num):
    ch = f"Chapter {chapter_num}"
    p1 = os.path.join(MANGA_FOLDER,manga_name)
    path = os.path.join(p1,ch)
    fn = f'/content/{ch}.cbr'
    downloader(fn,url)
    print(f"size: {get_file_size_mb(fn)}")
    unzip(fn,path)
    #$patoolib.extract_archive(archive=fn,verbosity=0,outdir=path)
    
  else:
    print("Check inputs")

try:
  download_zip()
except Exception as e:
  print(str(e))

In [ ]:
#@title Download Manga
#@markdown MangaPill Chapter URL
MANGA_CHAP_URL = "https://mangapill.com/chapters/5061-10005000/kimi-ga-shinu-made-ato-100-nichi-chapter-5" # @param {type:"string"}

# DOWNLOAD = True # @param {type:"boolean"}
# COLOURIZE = True # @param {type:"boolean"}
# MAKE_CBZ = True # @param {type:"boolean"}


import os
import patoolib
from IPython.display import clear_output

URL_MANGA_PILL = 'https://mangapill.com'

def downloadMangaPill(link):
    hdr = {"Referer":f"{URL_MANGA_PILL}/"}
    # chp_name = chapter.text

    # Load the chapter
    c = requests.get(link)

    if c.status_code == requests.codes.ok:
      chapter_parser = BeautifulSoup(c.text, 'html.parser')
      title = chapter_parser.title
      title_split = title.text.split(" Chapter ")
      manga_name = title_split[0].title()
      CHAP_NAME = "Chapter "+title_split[1].replace(" - Mangapill",'')

      MANGA_PATH = MANGA_FOLDER+"/"+manga_name + '/' + CHAP_NAME
      mkdir(MANGA_PATH)
      MANGA_DETAILS = {'chap':CHAP_NAME,'manga':manga_name}
      pages = chapter_parser.find_all('img', attrs={'class': 'js-page'})

      for num,page in enumerate(pages):
        num+=1
        img_url = page['data-src']
          # filenm = chp_folder + '/' + url.split('/')[-1]
        a = urlparse(img_url)
        # fileName = "00"+os.path.basename(a.path)
        fileName,ext = os.path.splitext(a.path)
        fileName = "{:05d}{ext}".format(num,ext=ext)
        down_path = MANGA_PATH+"/"+fileName
        downloader(down_path,img_url,hdr)
      clear_output()
      print(f"Downloaded at {MANGA_PATH} - {num} files")


if URL_MANGA_PILL in MANGA_CHAP_URL:
  downloadMangaPill(MANGA_CHAP_URL)


#Process

In [ ]:
#@title  Colourize
MANGA_PATH = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/" # @param {type:"string"}
#@markdown ---
#@markdown Colourize Config
GENERATOR = "networks/generator.zip" # @param {type:"string"}
EXTRACTOR = "networks/extractor.pth" # @param {type:"string"}
GPU = True# @param {type:"boolean"}
DENOISE = True# @param {type:"boolean"}
DENOISE_SIGMA = 25# @param {type:"integer"}
#@markdown default SIZE_FACTOR = 18
SIZE_FACTOR = 20# @param {type:"integer"}

#@markdown ---

cur_dir=os.getcwd()
if cur_dir!="/content/manga-colorization-v2":
  os.chdir(PROJECT_NAME)
from inference import colorize_single_image,convert_to_bw
from colorizator import MangaColorizator
import matplotlib.pyplot as plt
from IPython.display import clear_output

SIZE = SIZE_FACTOR*32
device = 'cpu'
if GPU:
  device = 'cuda'

temp_name = "colour"


def process_image(image, colorizator):
    colorizator.set_image(image, SIZE, DENOISE, DENOISE_SIGMA)
    return colorizator.colorize()

def color_it(image_path,save_path):
  convert_to_bw(image_path)
  image = plt.imread(image_path)
  colorizer = MangaColorizator(device,GENERATOR,EXTRACTOR)

  colorization = process_image(image, colorizer,)
  plt.imsave(save_path, colorization)


try:
  if os.path.isdir(MANGA_PATH):
      SAVE_PATH = os.path.join(MANGA_PATH,temp_name)
      mkdir(SAVE_PATH)
      num = 0
      for img in os.listdir(MANGA_PATH):
        image_path = os.path.join(MANGA_PATH,img)
        save_path = os.path.join(SAVE_PATH,img)
        if not os.path.isfile(image_path):
          continue
        if "_colour" in img or ".xml" in img:
          continue
        num+=1
        color_it(image_path,save_path)
        print(f"SAVED: {save_path}")
      clear_output()
      print(f"SAVED: {SAVE_PATH} - {num} files")

  elif os.path.isfile(MANGA_PATH):
    fileName,ext = os.path.splitext(MANGA_PATH)
    save_path = f"{fileName}_{temp_name}{ext}"

    color_it(MANGA_PATH,save_path)

    print(f"SAVED: {save_path}")
except Exception as e:
  print(str(e))
os.chdir("..")

SAVED: /content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/colour - 16 files


In [ ]:
#@title Upscaler
import os,shutil
#@markdown Folder that contains all the images
MANGA_PATH = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/colour" # @param {type:"string"}
#@markdown default "SCALE_FACTOR" = 4
SCALE_FACTOR = "2" # @param ["2", "3", "4"]
# MODEL_NAME = "RealESRGAN_x4plus" # @param {type:"string"}


UPSCALER_DIR = "Real-ESRGAN"
os.chdir(UPSCALER_DIR)
upsc_folder = "upscaled"

try:
  def upscale_image(img):
    og_folder,og_filename = os.path.split(img)

    folder = os.path.join(og_folder,upsc_folder)

    cmd = [
      'python',
      'inference_realesrgan.py',
      f'-n "{upscaler_model_name}" ',
      f'-i "{img}" ',
      f'-o "{folder}" '
      '--outscale',
      f'{SCALE_FACTOR}',
    ]
    cmd_str = ' '.join(cmd)
    os.system(cmd_str)
    name,ext = os.path.splitext(og_filename)
    save_path = os.path.join(folder,f"{name}_out{ext}")
    print(f"DONE: {save_path}")

  if os.path.isfile(MANGA_PATH):
    upscale_image(MANGA_PATH)
  else:
    for img in os.listdir(MANGA_PATH):
      file_path = os.path.join(MANGA_PATH,img)
      if os.path.isfile(file_path):
        upscale_image(file_path)

except Exception as e:
  print(str(e))
os.chdir("..")

In [ ]:
#@title Compress images
#@markdown single image or folder path
PATH = "/content/Manga/Kagurabachi/Chapter 23" # @param {type:"string"}
QUALITY = 80 # @param {type:"integer"}

from io import BytesIO
import os
from PIL import Image
from IPython.display import clear_output


def compress(img_path,out_img_path):

  img = Image.open(img_path)

  buffer = BytesIO()
  img.save(buffer, "JPEG", quality=int(QUALITY))
  
  f = open(out_img_path, "wb")
  f.write(buffer.getbuffer())
  f.close()
  print(f"DONE: {out_img_path}")




final_out = ""
if os.path.isfile(PATH):
  name,ext = os.path.splitext(PATH)
  out_img_path = f"{name}_q{QUALITY}{ext}"
  compress(PATH,out_img_path)
  final_out = out_img_path

else:
  out_path = os.path.join(PATH,"compress")
  mkdir(out_path)
  final_out = out_path
  # print(out_path)
  for img_name in os.listdir(PATH):
    img_path = os.path.join(PATH,img_name)
    if os.path.isfile(img_path):
      out_img_path = os.path.join(out_path,img_name)
      compress(img_path,out_img_path)
      # clear_output()

clear_output()
print(f"DONE: {final_out}")

In [ ]:
#@title Build CBZ
IMAGES_FOLDER = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/colour/upscaled/" # @param {type:"string"}
ZIP_FILENAME = "Chapter 5.cbz" # @param {type:"string"}



import patoolib
import shutil,os
from IPython.display import clear_output


zip_files = []
if os.path.isdir(IMAGES_FOLDER):
  for img in os.listdir(IMAGES_FOLDER):
    save_path = os.path.join(IMAGES_FOLDER,img)
    if os.path.isdir(save_path):continue
    zip_files.append(save_path)

  zip_filename = ZIP_FILENAME if len(ZIP_FILENAME) else "manga.cbz"
  zip_filename = os.path.join(IMAGES_FOLDER,zip_filename)
  patoolib.create_archive(zip_filename, zip_files, verbosity = 1, interactive = False)
  printLine()
  print(f"SAVED: {zip_filename}" )
else:
  print("Provide directory")

In [ ]:
# @title Send To Telegram
import requests,json
from telegram import Bot
from telegram.error import TelegramError

BOT_TOKEN = "" # @param {type:"string"}
CHANNEL_ID = "" # @param {type:"string"}
DOC = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/colour/Chapter 5.cbz" # @param {type:"string"}
CAPTION = "Kimi Ga Shinu Made Ato 100 Nichi - Chapter 5" # @param {type:"string"}

if len(BOT_TOKEN) and len(CHANNEL_ID) and len(DOC):
  text = CAPTION
  zip_filename = DOC
  print(f"SENDING: {zip_filename}")

  bot = Bot(token=BOT_TOKEN)
  try:
    await bot.send_document(chat_id=CHANNEL_ID, document=open(zip_filename, 'rb'),caption=text)
    print(f'SENT: {text}')
  except TelegramError as e:
      print(f'Failed to send document: {e}')

else:
  print("Check token and channel id and doc")

In [ ]:
#@title Copy To Drive
import os,shutil
from google.colab import drive

SRC_FILE = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 1/colour/Chapter 1.cbz" # @param {type:"string"}
DEST_FOLDER = "Kimi Ga Shinu Made Ato 100 Nichi" # @param {type:"string"}

DRIVE = "/content/drive/"
if not os.path.exists(DRIVE):
  drive.mount(DRIVE)

DRIVE_MANGA = os.path.join('/MyDrive/Manga/',DEST_FOLDER)
if not DRIVE in DEST_FOLDER:
  DRIVE_MANGA = os.path.join(DRIVE,DRIVE_MANGA)
mkdir(DRIVE_MANGA)

if os.path.exists(SRC_FILE):
  shutil.copy(SRC_FILE,DRIVE_MANGA)
  print(f"COPIED: to {DRIVE_MANGA}")
else:
  print(f"FAILED: {SRC_FILE} doesnt exists")



In [ ]:
# @title View Image
img_path = "/content/Manga/Kimi Ga Shinu Made Ato 100 Nichi/Chapter 5/00001.jpeg" # @param {type:"string"}

import cv2
from google.colab.patches import cv2_imshow
# Open the image.
img = cv2.imread(img_path)
print(f"h,w : {img.shape[:2]}")
print(f"size: {get_file_size_mb(img_path)} mb")
cv2_imshow(img)